<a href="https://colab.research.google.com/github/itsniranjan/ShipTruckClassifier/blob/main/BinaryClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


###Here, we'll take CIFAR10 dataset and extract the ship and truck images from it

In [2]:
(trainX, trainY),(testX,testY)=datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
trainY=trainY.reshape(-1,)
testY=testY.reshape(-1,)

In [5]:
trainX=trainX/255
testX=testX/255

In [6]:
shipcount=0
truckcount=0
for y in trainY:
  if(y==8):
    shipcount+=1
  elif(y==9):
    truckcount+=1
print(shipcount)
print(truckcount)

5000
5000


In [7]:
STtrainX=[]
STtrainY=[]

for index in range(50000):
  if(trainY[index]==9 or trainY[index]==8):
    STtrainX.append(trainX[index])
    STtrainY.append(trainY[index])
print(len(STtrainX))


10000


In [8]:
STtestX=[]
STtestY=[]

for index in range(10000):
  if(testY[index]==9 or testY[index]==8):
    STtestX.append(testX[index])
    STtestY.append(testY[index])
print(len(STtestX))

2000


In [9]:
for _ in range(len(STtrainY)):
  if(STtrainY[_]==8):
    STtrainY[_]=0
  else:
    STtrainY[_]=1
for _ in range(len(STtestY)):
  if(STtestY[_]==8):
    STtestY[_]=0
  else:
    STtestY[_]=1
  
print(STtrainY[:5])
print(STtestY[:5])


[1, 1, 0, 1, 1]
[0, 0, 1, 1, 0]


In [10]:
# ann=models.Sequential([
#                        layers.Flatten(input_shape=(32,32,3)),
#                         layers.Dense(3000,activation='relu'),
#                         layers.Dense(2,activation='softmax')
# ])

In [11]:
# ann.compile(optimizer='SGD',
#             loss='sparse_categorical_crossentropy',
#             metrics=['accuracy']
#           )

In [12]:
# ann.fit(np.array(STtrainX),np.array(STtrainY),epochs=10)

In [13]:
# ann.evaluate(np.array(STtrainX),np.array(STtrainY))

In [14]:
cnn= models.Sequential([
                        layers.Conv2D(filters=5,kernel_size=(3,3),activation='relu',input_shape=(32,32,3)),
                        layers.MaxPooling2D((2,2)),
                        layers.Conv2D(filters=5,kernel_size=(3,3),activation='relu'),
                        layers.MaxPooling2D((2,2)),
                        layers.Flatten(),
                        layers.Dense(10,activation='relu'),
                        layers.Dense(2,activation='softmax')
])


In [15]:
cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics='accuracy'
          )

In [16]:
cnn.fit(np.array(STtrainX),np.array(STtrainY),epochs=10)

Epoch 1/10
313/313 [==============================] - 6s 18ms/step - loss: 0.5464 - accuracy: 0.7228
Epoch 2/10
313/313 [==============================] - 6s 18ms/step - loss: 0.3944 - accuracy: 0.8262
Epoch 3/10
313/313 [==============================] - 5s 18ms/step - loss: 0.3546 - accuracy: 0.8476
Epoch 4/10
313/313 [==============================] - 5s 17ms/step - loss: 0.3401 - accuracy: 0.8523
Epoch 5/10
313/313 [==============================] - 6s 18ms/step - loss: 0.3179 - accuracy: 0.8672
Epoch 6/10
313/313 [==============================] - 6s 18ms/step - loss: 0.3030 - accuracy: 0.8724
Epoch 7/10
313/313 [==============================] - 6s 18ms/step - loss: 0.2912 - accuracy: 0.8803
Epoch 8/10
313/313 [==============================] - 6s 18ms/step - loss: 0.2846 - accuracy: 0.8830
Epoch 9/10
313/313 [==============================] - 6s 18ms/step - loss: 0.2707 - accuracy: 0.8880
Epoch 10/10
313/313 [==============================] - 6s 18ms/step - loss: 0.2582 - accura

In [17]:
cnn.evaluate(np.array(STtrainX),np.array(STtrainY))

313/313 [==============================] - 3s 9ms/step - loss: 0.2850 - accuracy: 0.8769


[0.28499728441238403, 0.8769000172615051]

##with cnn, we got an accuracy of 0.909

In [18]:

tf.keras.models.save_model(cnn,'mymodel.hdf5')

In [9]:
!pip install -U ipykernel
!pip install -q streamlit

In [10]:
%%writefile app.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Ship Truck Classifie")
st.text("Provide URL of  Image for image classification")

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/mymodel.hdf5')
  return model
with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['ship','truck']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return tf.image.resize(image,[32,32])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

# path = st.text_input('Enter Image URL to Classify.. ','https://cdn.britannica.com/17/126517-050-9CDCBDDF/semi-semitrailer-truck-tractor-highway.jpg')
# if path is not None:
#     content = requests.get(path).content

#     st.write("Predicted Class :")
#     with st.spinner('classifying.....'):
#       label =np.argmax(model.predict(decode_img(content)),axis=1)
#       st.write(classes[label[0]])    
#     st.write("")
#     image = Image.open(BytesIO(content))
#     st.image(image, caption='Classifying Image', use_column_width=True)

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):
    
        size = (32,32)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    st.write(prediction)
    st.write(score)
    print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(score))
)

Overwriting app.py


In [11]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip


--2021-08-11 21:53:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.82.179.106, 52.44.51.219, 54.243.172.172, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.82.179.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  18.8MB/s    in 0.7s    

2021-08-11 21:53:29 (18.8 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]



In [13]:
!unzip ngrok-stable-linux-amd64.zip


Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [14]:
get_ipython().system_raw('./ngrok http 8501 &')


In [15]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://9a8ff4faa938.ngrok.io


In [ ]:

!streamlit run /content/app.py

2021-08-11 21:53:52.930935: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.223.98:8501

2021-08-11 21:53:54.395 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/caching.py", line 515, in get_or_create_cached_value
    hash_funcs=hash_funcs,
  File "/usr/local/lib/python3.7/dist-packages/streamlit/caching.py", line 308, in _read_from_cache
    raise e
  File "/usr/local/lib/python3.7/dist-packages/streamlit/caching.py", line 294, in _read_from_cache
    mem_cache, key, allow_output_mutation, func_or_code, hash_funcs
  File "/usr/local/lib/python3.7/dist-packages/streamlit/caching.py", line 212, in _read_from_mem_cache
    raise CacheKeyNotFoundError("Key not found in mem cache")
streamlit.caching.CacheKeyNotFoundError: Key not found in mem cache

Durin

##Please bare any issues with ngrok